# Multiple Results


In this notebook we show how you can in a single request, have the LLM model return multiple results per prompt. This is useful for running experiments where you want to evaluate the robustness of your prompt and the parameters of your config against a particular large language model.


In [ ]:
!python -m pip install semantic-kernel==1.0.5

In [3]:
import sys
import os

# Add the samples directory to the Python path
samples_dir = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples'
sys.path.append(samples_dir)

try:
    from services import Service
    from service_settings import ServiceSettings

    service_settings = ServiceSettings(
        global_llm_service="AzureOpenAI")  # Example setting

    # Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
    selectedService = (
        Service.AzureOpenAI
        if service_settings.global_llm_service is None
        else Service(service_settings.global_llm_service.lower())
    )
    print(f"Using service type: {selectedService}")
except ModuleNotFoundError as e:
    print(f"Error: {e}")
    print("Ensure that 'service_settings.py' is in the correct directory and properly named.")
    # Additional debugging information
    print("Current Python path:")
    for path in sys.path:
        print(path)

Using service type: Service.AzureOpenAI


In [4]:
from dotenv import load_dotenv
from semantic_kernel import Kernel

kernel = Kernel()


env_path = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples\getting_started\.env'
load_dotenv(env_path)

service_id = None

# Load the API keys and other settings from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_org_id = os.getenv('OPENAI_ORG_ID')
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_chat_deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')


if selectedService == Service.OpenAI:

    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"

    kernel.add_service(


        OpenAIChatCompletion(service_id=service_id,
                             ai_model_id="gpt-3.5-turbo-1106"),


    )


elif selectedService == Service.AzureOpenAI:

    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"

    kernel.add_service(


        AzureChatCompletion(
            service_id=service_id,
            api_key=azure_openai_api_key,           # Use the api_key from the .env file
            # Use the deployment_name from the .env file
            deployment_name=azure_chat_deployment_name,
            endpoint=azure_openai_endpoint,         # Use the endpoint from the .env file
            env_file_path=env_path,




        )
    )

First, we will set up the text and chat services we will be submitting prompts to.


In [13]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.hugging_face import (  # noqa: F401
    HuggingFacePromptExecutionSettings,
    HuggingFaceTextCompletion,
)
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings  # noqa: F401
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings  # noqa: F401
from semantic_kernel.connectors.ai.open_ai import OpenAITextPromptExecutionSettings  # noqa: F401
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextCompletion,
    OpenAIChatCompletion,
    OpenAITextCompletion,
)
from dotenv import load_dotenv

env_path = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples\getting_started\.env'
load_dotenv(env_path)

service_id = None

# Load the API keys and other settings from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_org_id = os.getenv('OPENAI_ORG_ID')
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_chat_deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')

kernel = Kernel()

# Configure Azure LLM service
if selectedService == Service.AzureOpenAI:
    azure_text_service = AzureTextCompletion(
        service_id="aoai_text",
        api_key=azure_openai_api_key,           # Use the api_key from the .env file
        # Use the deployment_name from the .env file
        deployment_name=azure_chat_deployment_name,
        endpoint=azure_openai_endpoint,         # Use the endpoint from the .env file
        env_file_path=env_path,
    )  # set the deployment name to the value of your text model (e.g. gpt-35-turbo-instruct)
    azure_chat_service = AzureChatCompletion(
        service_id="aoai_chat",
        api_key=azure_openai_api_key,           # Use the api_key from the .env file
        # Use the deployment_name from the .env file
        deployment_name=azure_chat_deployment_name,
        endpoint=azure_openai_endpoint,         # Use the endpoint from the .env file
        env_file_path=env_path,
    )  # set the deployment name to the value of your chat model

# Configure OpenAI service
if selectedService == Service.OpenAI:
    oai_text_service = OpenAITextCompletion(service_id="oai_text", ai_model_id="gpt-3.5-turbo-instruct")
    oai_chat_service = OpenAIChatCompletion(service_id="oai_chat", ai_model_id="gpt-3.5-turbo")

# Configure Hugging Face service
if selectedService == Service.HuggingFace:
    hf_text_service = HuggingFaceTextCompletion(service_id="hf_text", ai_model_id="distilgpt2", task="text-generation")

Next, we'll set up the completion request settings for text completion services.


In [14]:
oai_text_prompt_execution_settings = OpenAITextPromptExecutionSettings(
    service="oai_text",
    extension_data={
        "max_tokens": 80,
        "temperature": 0.7,
        "top_p": 1,
        "frequency_penalty": 0.5,
        "presence_penalty": 0.5,
        "number_of_responses": 3,
    },
)

## Multiple Open AI Text Completions


In [15]:
if selectedService == Service.OpenAI:
    prompt = "What is the purpose of a rubber duck?"

    results = await oai_text_service.get_text_contents(prompt=prompt, settings=oai_text_prompt_execution_settings)
    i = 1
    for result in results:
        print(f"Result {i}: {result}")
        i += 1

## Multiple Azure Open AI Text Completions


In [16]:
if selectedService == Service.AzureOpenAI:
    prompt = "provide me a list of possible meanings for the acronym 'ORLD'"

    results = await azure_text_service.get_text_contents(prompt=prompt, settings=oai_text_prompt_execution_settings)
    i = 1
    for result in results:
        print(f"Result {i}: {result}")
        i += 1

Result 1:  and I'll select the 'best' one. #acronyms

@EriktheViking12 @johncleese I'm not sure. Maybe it's a mix of "buffoonery" and "clownishness" which would be an apt description of #BorisJohnson?

What's the correct term for a "female #swashbuckler"? Is
Result 2:  6 days left

Looking for a computer programmer to provide me with a list of possible meanings for the acronym 'ORLD' . This is for a brainstorming session with a client in the healthcare sector, and we need to come up with as many words as possible for this acronym. Letters within the acronym can be used more than once, and each letter doesn't necessarily have to be the start of
Result 3:  that are not already associated with any major organizations.

Subject: Re: ORLD acronyms

Answered By: robertskelton-ga on 17 Apr 2003 05:01 PDT

Rated:

Hello, 

Here are some acronyms for "ORLD" which seem to be unclaimed. I have
also added some possible meanings (some of them a little far


## Multiple Hugging Face Text Completions


In [17]:
if selectedService == Service.HuggingFace:
    hf_prompt_execution_settings = HuggingFacePromptExecutionSettings(
        service_id="hf_text", extension_data={"max_new_tokens": 80, "temperature": 0.7, "top_p": 1}
    )

In [18]:
if selectedService == Service.HuggingFace:
    prompt = "The purpose of a rubber duck is"

    results = await hf_text_service.get_text_contents(
        prompt=prompt, prompt_execution_settings=hf_prompt_execution_settings
    )
    print("".join(results))

Here, we're setting up the settings for Chat completions.


In [19]:
oai_chat_prompt_execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="oai_chat",
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3,
)

## Multiple OpenAI Chat Completions


In [20]:
from semantic_kernel.contents import ChatHistory

if selectedService == Service.OpenAI:
    chat = ChatHistory()
    chat.add_user_message(
        "It's a beautiful day outside, birds are singing, flowers are blooming. On days like these, kids like you..."
    )
    results = await oai_chat_service.get_chat_message_contents(
        chat_history=chat, settings=oai_chat_prompt_execution_settings
    )
    i = 0
    for result in results:
        print(f"Result {i + 1}: {result!s}")
        i += 1

## Multiple Azure OpenAI Chat Completions


In [21]:
az_oai_prompt_execution_settings = AzureChatPromptExecutionSettings(
    service_id="aoai_chat",
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3,
)

In [22]:
if selectedService == Service.AzureOpenAI:
    content = (
        "Tomorrow is going to be a great day, I can feel it. I'm going to wake up early, go for a run, and then..."
    )
    chat = ChatHistory()
    chat.add_user_message(content)
    results = await azure_chat_service.get_chat_message_contents(
        chat_history=chat, settings=az_oai_prompt_execution_settings
    )
    i = 0
    for result in results:
        print(f"Result {i + 1}: {result!s}")
        i += 1

Result 1: enjoy a healthy breakfast. I'll have some fresh fruit, yogurt, and granola to fuel up for the day ahead. After that, I'll spend some time meditating and setting my intentions for the day.

Once I'm feeling centered and focused, I'll start tackling my to-do list. I have a few big projects at work that I've been working on, so I'll dive
Result 2: I'm going to tackle all of the tasks on my to-do list with enthusiasm and focus. I'll start with some work projects that have been on my mind, then move onto some personal errands like grocery shopping and cleaning. In the evening, I plan to meet up with some friends for dinner and enjoy good company and conversation. Overall, I'm excited for a productive and fulfilling day ahead.
Result 3: I'm going to have a delicious breakfast with my family and spend some quality time with them. After that, I have planned a fun day out with my friends, where we will go hiking and have a picnic in the park. In the evening, I'm going to attend a conc

## Streaming Multiple Results

Here is an example pattern if you want to stream your multiple results. Note that this is not supported for Hugging Face text completions at this time.


In [23]:
if selectedService == Service.OpenAI:
    import os
    import time

    from IPython.display import clear_output

    # Determine the clear command based on OS
    clear_command = "cls" if os.name == "nt" else "clear"

    chat = ChatHistory()
    chat.add_user_message("what is the purpose of a rubber duck?")

    stream = oai_chat_service.get_streaming_chat_message_contents(
        chat_history=chat, settings=oai_chat_prompt_execution_settings
    )
    number_of_responses = oai_chat_prompt_execution_settings.number_of_responses
    texts = [""] * number_of_responses

    last_clear_time = time.time()
    clear_interval = 0.5  # seconds

    # Note: there are some quirks with displaying the output, which sometimes flashes and disappears.
    # This could be influenced by a few factors specific to Jupyter notebooks and asynchronous processing.
    # The following code attempts to buffer the results to avoid the output flashing on/off the screen.

    async for results in stream:
        current_time = time.time()

        # Update texts with new results
        for idx, result in enumerate(results):
            if idx < number_of_responses:
                texts[idx] += str(result)

        # Clear and display output at intervals
        if current_time - last_clear_time > clear_interval:
            clear_output(wait=True)
            for idx, text in enumerate(texts):
                print(f"Result {idx + 1}: {text}")
            last_clear_time = current_time

    print("----------------------------------------")